In [3]:
using StaticArrays
import Base: +, *
using Distributions , Plots
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /srv/julia/pkg/packages/Plots/S2aH5/src/backends.jl:374


Plots.PlotlyBackend()

In [4]:
struct MultiDual{N,T} 
    val::T
    derivs::SVector{N,T}
end

In [5]:
function Base.:+(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T} 
        return MultiDual{N,T}(f.val + g.val, f.derivs + g.derivs)     
    end

    function Base.:-(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T} 
        return MultiDual{N,T}(f.val - g.val, f.derivs - g.derivs)
    end
    
    function Base.:*(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T} 
        return MultiDual{N,T}(f.val * g.val, f.val .* g.derivs + g.val .* f.derivs)
    end

    function Base.:*(f::MultiDual{N,T}, α::Number) where {N,T} 
        return MultiDual{N,T}(f.val * α, f.derivs .* α)
    end

    Base.:^(f::MultiDual{N, T}, n::Integer) where {N,T} = Base.power_by_squaring(f, n)

    Base.:*(α::Number, f::MultiDual{N, T}) where {N, T} = f*α 

In [6]:
Base.show(io::IO,x::MultiDual) = print(io,x.val," + ",x.derivs," ε")

In [7]:
a=MultiDual(2.0,SVector(1.0,0.0))

2.0 + [1.0, 0.0] ε

In [8]:
b=MultiDual(4.0,SVector(1.0,0.0))

4.0 + [1.0, 0.0] ε

In [9]:
function Base.:log(f::MultiDual{N,T}) where {N,T}    #Log
        return MultiDual(log(f.val), inv(f.val) .*f.derivs)
    end
    
    function Base.:exp(f::MultiDual{N,T}) where {N,T}    #expo
            return MultiDual(exp(f.val), exp(f.val)*f.derivs)      
        end
    
    function Base.:sin(f::MultiDual{N,T}) where {N,T}     #sin
        return MultiDual(sin(f.val),cos(f.val)*f.derivs)
    end

    function Base.:cos(f::MultiDual{N,T}) where {N,T}     #cos
        return MultiDual(cos(f.val),-sin(f.val)*f.derivs)
    end
    
    function Base.:^(f::MultiDual{N,T} , a ::Real) where {N,T}     #power
            return MultiDual((f.val)^a,(a)*((f.val)^(a-1))*f.derivs)   
        end
    
    function Base.:abs(f::MultiDual{N,T}) where {N,T}    #abs
            return MultiDual(abs(f.val),f.derivs)                    
        end
    
    function Base.:/(f::MultiDual{N,T} , g::MultiDual{N,T}) where {N,T}    #devide
            return MultiDual(f.val/g.val,(f.derivs*g.val-f.val*g.derivs)/g.val*g.val)
        end

In [10]:
log(a)

0.6931471805599453 + [0.5, 0.0] ε

In [11]:
exp(a)^2

54.59815003314424 + [109.19630006628849, 0.0] ε

In [12]:
cos(a)

-0.4161468365471424 + [-0.9092974268256817, -0.0] ε

In [13]:
sin(a)

0.9092974268256817 + [-0.4161468365471424, -0.0] ε

In [14]:
x = MultiDual(2.0, SVector(1.0,0.0))
    log(x)

0.6931471805599453 + [0.5, 0.0] ε

In [15]:
h(x,y)=(log(x)+x*exp(y))

h (generic function with 1 method)

In [16]:
h(a,b)

109.88944724684842 + [164.2944500994327, 0.0] ε

In [17]:
g(x,y)=(x*y,x^2+y^3,sin(x^2))

g (generic function with 1 method)

In [18]:
g(a,b)

(8.0 + [6.0, 0.0] ε, 68.0 + [52.0, 0.0] ε, -0.7568024953079282 + [-2.6145744834544478, -0.0] ε)